In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle as skshuffle
from sklearn.multiclass import OneVsRestClassifier
from scipy import sparse

In [50]:
from scipy.io import loadmat
from scipy.io import savemat
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist
from scipy.spatial.distance import cdist

In [55]:

mat = loadmat('./results_dblp_4.mat')
features_matrix = mat['embs']
#print(type(features_matrix))
#print(features_matrix[:10,:10])

mat = loadmat("../data/dblp.mat")
A = mat["network"]
#graph = sparse2graph(A)
labels_matrix = mat["group"]
labels_count = labels_matrix.shape[1]

print(labels_matrix)

print(type(labels_matrix))

groups = labels_matrix.toarray()

print(type(groups[:5, :5]))
print(groups[:5, :5])

  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 0)	1.0
  (5, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (11, 0)	1.0
  (14, 0)	1.0
  (15, 0)	1.0
  (18, 0)	1.0
  (19, 0)	1.0
  (20, 0)	1.0
  (21, 0)	1.0
  (22, 0)	1.0
  (23, 0)	1.0
  (24, 0)	1.0
  (25, 0)	1.0
  (28, 0)	1.0
  (30, 0)	1.0
  (31, 0)	1.0
  (32, 0)	1.0
  (33, 0)	1.0
  :	:
  (12806, 1)	1.0
  (12808, 1)	1.0
  (12809, 1)	1.0
  (12810, 1)	1.0
  (12811, 1)	1.0
  (12812, 1)	1.0
  (12815, 1)	1.0
  (12817, 1)	1.0
  (12818, 1)	1.0
  (12820, 1)	1.0
  (12821, 1)	1.0
  (12823, 1)	1.0
  (12827, 1)	1.0
  (12829, 1)	1.0
  (12830, 1)	1.0
  (12831, 1)	1.0
  (12832, 1)	1.0
  (12833, 1)	1.0
  (12834, 1)	1.0
  (12835, 1)	1.0
  (12837, 1)	1.0
  (12838, 1)	1.0
  (12839, 1)	1.0
  (12840, 1)	1.0
  (12841, 1)	1.0
<class 'scipy.sparse._csc.csc_matrix'>
<class 'numpy.ndarray'>
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 1.]
 [1. 0.]]


In [56]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, gram_test, top_k_list):
        assert gram_test.shape[0] == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(gram_test))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            all_labels.append(labels)
        return all_labels

In [61]:
# Example data creation
n = 13326  # Number of items
num_groups = 2  # Number of groups
#groups = np.random.randint(0, 2, size=(n, num_groups))  # Random binary groups matrix
print(type(groups))
#groups = sparse.random(n, num_groups, density=0.2, format='csr', data_rvs=np.ones).toarray()  # Sparse binary groups matrix with 20% density
features = np.random.rand(n, 128)  # Random features in 128 columns

print(groups[:5, :5])

# Creating DataFrame for features
df_features = pd.DataFrame(features, columns=[f'feature_{i}' for i in range(128)])

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(df_features, groups, test_size=0.3, random_state=42)

clf = TopKRanker(SVC(kernel="precomputed",cache_size=4096,probability=True))
gram = squareform( 1 - pdist(X_train, 'hamming'));
gram_test = 1 - cdist(X_test, X_train, 'hamming');

clf.fit(gram, y_train)

top_k_list = [len(l) for l in y_test]
preds = clf.predict(gram_test, top_k_list)
mlb = MultiLabelBinarizer(classes=range(2))
f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average='micro')

accuracy = accuracy_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), normalize=False) / y_test.size
recalls = recall_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=None)  # Returns recall for each class
f1s = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=None)  # Returns F1 score for each class

print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Average Recall: {np.mean(recalls):.2f}")
print(f"Average F1 Score: {np.mean(f1s):.2f}")
print("\nClassification Report for each group:")
for i in range(num_groups):
    print(f"\nGroup {i} Classification Report:")
    print(classification_report(y_test[:, i], preds[:, i]))

<class 'numpy.ndarray'>
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 1.]
 [1. 0.]]
Overall Accuracy: 0.49
Average Recall: 1.00
Average F1 Score: 1.00

Classification Report for each group:

Group 0 Classification Report:


TypeError: list indices must be integers or slices, not tuple

In [58]:

# Model setup - MultiOutput with Logistic Regression
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))

# Model training
model.fit(X_train, y_train)

# Predicting
y_pred = model.predict(X_test)

# Converting sparse matrix to dense for evaluation if necessary (depends on your specific evaluation function)
y_test_dense = y_test#.toarray()
y_pred_dense = y_pred#.toarray()

# Evaluating the model using example metrics (adjust according to your specific needs)
accuracy = accuracy_score(y_test_dense, y_pred_dense, normalize=False) / y_test_dense.size
recalls = recall_score(y_test_dense, y_pred_dense, average=None)  # Returns recall for each class
f1s = f1_score(y_test_dense, y_pred_dense, average=None)  # Returns F1 score for each class

print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Average Recall: {np.mean(recalls):.2f}")
print(f"Average F1 Score: {np.mean(f1s):.2f}")
print("\nClassification Report for each group:")
for i in range(num_groups):
    print(f"\nGroup {i} Classification Report:")
    print(classification_report(y_test_dense[:, i], y_pred_dense[:, i]))
#df_group.head()

Overall Accuracy: 0.26
Average Recall: 0.53
Average F1 Score: 0.47

Classification Report for each group:

Group 0 Classification Report:
              precision    recall  f1-score   support

         0.0       0.47      0.11      0.18      1752
         1.0       0.57      0.90      0.70      2246

    accuracy                           0.56      3998
   macro avg       0.52      0.51      0.44      3998
weighted avg       0.52      0.56      0.47      3998


Group 1 Classification Report:
              precision    recall  f1-score   support

         0.0       0.55      0.85      0.67      2191
         1.0       0.47      0.16      0.24      1807

    accuracy                           0.54      3998
   macro avg       0.51      0.51      0.46      3998
weighted avg       0.52      0.54      0.48      3998



In [ ]:

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, num_groups:], df.iloc[:, :num_groups], test_size=0.3, random_state=42)

# Model setup - MultiOutput with Logistic Regression
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))

# Model training
model.fit(X_train, y_train)

# Predicting
y_pred = model.predict(X_test)

# Evaluating the model using example metrics (adjust according to your specific needs)
accuracy = accuracy_score(y_test, y_pred, normalize=False) / y_test.size
recalls = recall_score(y_test, y_pred, average=None)  # Returns recall for each class
f1s = f1_score(y_test, y_pred, average=None)  # Returns F1 score for each class

print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Average Recall: {np.mean(recalls):.2f}")
print(f"Average F1 Score: {np.mean(f1s):.2f}")
print("\nClassification Report for each group:")
for i in range(num_groups):
    print(f"\nGroup {i} Classification Report:")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

In [22]:
import numpy as np
from scipy.sparse import csc_matrix

# Create a small example csc_matrix
data = np.array([1, 2, 3, 4])
row_indices = np.array([0, 1, 2, 3])
col_indices = np.array([0, 1, 2, 3])
sparse_matrix = csc_matrix((data, (row_indices, col_indices)), shape=(4, 4))

# Convert csc_matrix to a dense ndarray
dense_array = sparse_matrix.toarray()

print("Dense ndarray from csc_matrix:")
print(dense_array[:3, :3])
print(sparse_matrix)

Dense ndarray from csc_matrix:
[[1 0 0]
 [0 2 0]
 [0 0 3]]
  (0, 0)	1
  (1, 1)	2
  (2, 2)	3
  (3, 3)	4
